In [1]:
import pandas as pd

In [2]:
rating_df = pd.read_csv("../dataset/ml-25m/ratings.csv")
rating_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [11]:
movies_df = pd.read_csv('../dataset/ml-25m/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Data preprocessing

In [5]:
rating_df.drop('timestamp', axis=1, inplace=True)
rating_df.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [7]:
rating_df.shape

(25000095, 3)

In [10]:
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 572.2 MB


In [12]:
movies_df.drop('genres', axis=1, inplace=True)
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [15]:
movies_df['title'] = movies_df.title.str.replace('\(\d\d\d\d\)','')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

/tmp/ipykernel_39546/2110638053.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('\(\d\d\d\d\)','')


,movieId,title
0,1,Toy Story
1,2,Jumanji
2,3,Grumpier Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


## Collaborative filtering

In [17]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [26]:
input_id

,movieId,title
0,1,Toy Story
1,2,Jumanji
292,296,Pulp Fiction
1241,1274,Akira
1879,1968,"Breakfast Club, The"
42774,164600,Akira


In [25]:
input_id = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(input_id, inputMovies)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,164600,Akira,4.5
5,1968,"Breakfast Club, The",5.0


In [27]:
userSubset = rating_df[rating_df.movieId.isin(inputMovies.movieId.tolist())]
userSubset.head()

,userId,movieId,rating
0,1,296,5.0
70,2,1,3.5
141,2,1968,1.0
254,3,1,4.0
264,3,296,5.0


In [28]:
userSubset.shape

(194464, 3)

In [31]:
userSubsetGroup = userSubset.groupby(['userId'])

In [34]:
userSubsetGroup.get_group(3)

,userId,movieId,rating
254,3,1,4.0
264,3,296,5.0
299,3,1274,4.5


In [35]:
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)

/tmp/ipykernel_39546/1784015714.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)


In [51]:
userSubsetGroup[0:3]

[(43,
        userId  movieId  rating
  5858      43        1     4.0
  5859      43        2     3.5
  5884      43      296     5.0
  5974      43     1274     4.5
  6018      43     1968     4.5),
 (171,
         userId  movieId  rating
  20794     171        1     4.5
  20795     171        2     4.0
  20833     171      296     5.0
  20973     171     1274     4.0
  21047     171     1968     4.0),
 (440,
         userId  movieId  rating
  54640     440        1     3.5
  54641     440        2     2.0
  54669     440      296     5.0
  54756     440     1274     5.0
  54798     440     1968     4.5)]

In [37]:
userSubsetGroup = userSubsetGroup[0:100]

In [54]:
from scipy.stats import pearsonr

In [63]:
inputMovies = inputMovies.sort_values(by='movieId')
temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
tempRatingList = temp_df['rating'].tolist()
tempRatingList

[3.5, 2.0, 5.0, 4.5, 5.0]

In [60]:
userSubsetGroup[0][1]

,userId,movieId,rating
5858,43,1,4.0
5859,43,2,3.5
5884,43,296,5.0
5974,43,1274,4.5
6018,43,1968,4.5


In [61]:
tempGroupList = userSubsetGroup[0][1]['rating'].tolist()
tempGroupList

[4.0, 3.5, 5.0, 4.5, 4.5]

In [62]:
corr, _ = pearsonr(tempRatingList, tempGroupList)
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.946


In [65]:
pearson_cor_dict = {}
for name, group in userSubsetGroup:
    group = group.sort_values(by='movieId')
    tempGroupList = group['rating'].tolist()
    corr, _ = pearsonr(tempRatingList, tempGroupList)
    pearson_cor_dict[name] = corr

/home/mohsen/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [67]:
pearson_cor_dict

{43: 0.9460287597114494,
 171: 0.328896757240136,
 440: 0.9615384615384615,
 597: 0.4688072309384954,
 695: 0.8770580193070293,
 757: 0.4385290096535146,
 772: 0.5847053462046861,
 860: 0.8594395636904107,
 939: 0.716114874039433,
 1203: -0.41671450165524826,
 1242: 0.7112333251538238,
 1401: 0.07520710469952338,
 1652: 0.5370861555295748,
 1894: 0.6020183016345587,
 1977: -0.3238751378156479,
 2345: 0.4385290096535146,
 2429: 0.4385290096535146,
 2469: 0.7161148740394329,
 2640: -0.5264497328966635,
 2766: -0.21926450482675733,
 2982: 0.7307692307692308,
 3150: -0.41602514716892186,
 3266: 0.657793514480272,
 3272: 0.0,
 3274: 0.17902871850985827,
 3624: 0.6698641270570836,
 3660: 0.775791113542719,
 3760: 0.5860090386731194,
 4006: 0.17902871850985827,
 4019: 0.39666441401095875,
 4060: 0.32328787506991996,
 4246: 0.8951435925492913,
 4410: 0.528885885347945,
 4420: 0.6577935144802719,
 4459: 0.8127665166512524,
 4675: 0.5370861555295748,
 4975: 0.9176462238110025,
 5114: -0.18605210

In [68]:
pearsonDF = pd.DataFrame.from_dict(pearson_cor_dict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.946029,43
1,0.328897,171
2,0.961538,440
3,0.468807,597
4,0.877058,695


In [69]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
99,0.983092,11495
64,0.970725,7723
55,0.962435,6550
94,0.961678,10960
63,0.961678,7571


In [71]:
topUsersRating=topUsers.merge(rating_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.983092,11495,1,3.0
1,0.983092,11495,2,1.0
2,0.983092,11495,6,4.5
3,0.983092,11495,10,4.5
4,0.983092,11495,16,4.5


In [72]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.983092,11495,1,3.0,2.949275
1,0.983092,11495,2,1.0,0.983092
2,0.983092,11495,6,4.5,4.423913
3,0.983092,11495,10,4.5,4.423913
4,0.983092,11495,16,4.5,4.423913


In [73]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,39.973168,152.887748
2,39.973168,98.535608
3,12.129644,31.955678
4,2.447945,5.662576
5,11.355650,23.548374


In [74]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.824759,1
2,2.465044,2
3,2.634511,3
4,2.313196,4
5,2.073714,5


In [75]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
71182,5.0,71182
55253,5.0,55253
108977,5.0,108977
5114,5.0,5114
6660,5.0,6660
1169,5.0,1169
299,5.0,299
96606,5.0,96606
6583,5.0,6583


In [76]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title
295,299,Priest
316,320,Suture
1141,1169,American Dream
5008,5114,"Bad and the Beautiful, The"
6460,6583,"Blood of Heroes, The (Salute of the Jugger, The)"
6537,6660,"Red Shoes, The"
11850,55253,"Lust, Caution (Se, jie)"
13741,71182,"Travelling Players, The (O thiasos)"
18452,96606,Samsara
21079,108977,Paranoia Agent
